In [1]:
from langchain import document_loaders as dl
from langchain import embeddings
from langchain import text_splitter as ts
from langchain import vectorstores as vs
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.runnable import RunnableParallel
from langchain.prompts import PromptTemplate
from operator import itemgetter

import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM

import re
import time

/home/aganap12/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import chromadb
from replicate.client import Client

replicate = Client(api_token="r8_IbZu0U3qUJ5ZC0TzzMm6xCJ6yU5UxVi16Ejo4")

client = chromadb.PersistentClient(path="./chromadb-client")
collection = client.get_or_create_collection("aws-blogs")


In [3]:
import os

def process_directory(directory_path):
    i = 1
    for filename in os.listdir(directory_path):
        # print(filename)
        file_path = os.path.join(directory_path, filename)
        if os.path.isfile(file_path):
            with open(file_path, "r", encoding="utf-8") as file:
                text = file.read()
                collection.add(
                documents=text,
                metadatas={"filename": filename},
                ids=f"doc{i}",
            )
            i += 1



directory_path = "../dataset/aws-case-studies-blogs-dataset"
process_directory(directory_path)

2024-04-17 19:06:41.268719392 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 3655288, index: 0, mask: {1, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2024-04-17 19:06:41.290222895 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 3655289, index: 1, mask: {2, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2024-04-17 19:06:41.310334302 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 3655290, index: 2, mask: {3, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2024-04-17 19:06:41.315109097 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 3655291, index: 3, mask: {4, }, error code: 22 error msg: Invalid argument. Specify the n

In [12]:
results = collection.query(
    query_texts=["DynamoDB"],
    n_results=5,
)

In [13]:
print(results)

{'ids': [['doc296', 'doc159', 'doc10', 'doc257', 'doc196']], 'distances': [[0.6806824207305908, 0.8134673833847046, 0.8582680821418762, 0.8845276832580566, 0.9456217885017395]], 'metadatas': [[{'filename': 'Improving Hiring Diversity and Accelerating App Development on AWS with Branch Insurance _ Case Study _ AWS.txt'}, {'filename': 'AWS Case Study - StreamAMG.txt'}, {'filename': 'Purple Technology Case Study _ AWS Step Functions.txt'}, {'filename': 'Game Studio Small Impact Games Runs Successful Alpha and Beta Tests Using Amazon GameLift _ Case Study _ AWS.txt'}, {'filename': 'Mobileye Cuts Costs Using Amazon EC2 _ Case Study _ AWS.txt'}]], 'embeddings': None, 'documents': [["Amazon DynamoDB is a fully managed, serverless, key-value NoSQL database designed to run high-performance applications at any scale. DynamoDB offers built-in security, continuous backups, automated multi-Region replication, in-memory caching, and data import and export tools.  Learn more\xa0»\nAmazon Cognito prov

In [10]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline

In [11]:
READER_MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)
#

llm = HuggingFacePipeline(pipeline=READER_LLM)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [14]:
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)
print(RAG_PROMPT_TEMPLATE)

<|system|>
Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}</s>
<|assistant|>



In [45]:
user_query = "How can I use Lambda?"
k=5
print(f"\nStarting retrieval for {user_query=}...")
print("\n==================================Top k documents==================================")
topk_retrieved_filename =  results['metadatas'][0][:k]
topk_retrieved_file_id =  results['ids'][0][:k]
topk_retrieved_file =  results['documents'][0][:k]
print(f"filename: {topk_retrieved_filename}\nfile_id: {topk_retrieved_file_id}")
print("\n==================================Top document==================================")
retrieved_filename =  results['metadatas'][0][0]
retrieved_file_id =  results['ids'][0][0]
retrieved_file =  results['documents'][0][0]
print(f"filename: {retrieved_filename['filename']}\nfile_id: {retrieved_file_id}")



Starting retrieval for user_query='How can I use Lambda?'...

==================================Top k documents==================================
filename: [{'filename': 'Improving Hiring Diversity and Accelerating App Development on AWS with Branch Insurance _ Case Study _ AWS.txt'}, {'filename': 'AWS Case Study - StreamAMG.txt'}, {'filename': 'Purple Technology Case Study _ AWS Step Functions.txt'}, {'filename': 'Game Studio Small Impact Games Runs Successful Alpha and Beta Tests Using Amazon GameLift _ Case Study _ AWS.txt'}, {'filename': 'Mobileye Cuts Costs Using Amazon EC2 _ Case Study _ AWS.txt'}]
file_id: ['doc296', 'doc159', 'doc10', 'doc257', 'doc196']

==================================Top document==================================
filename: Improving Hiring Diversity and Accelerating App Development on AWS with Branch Insurance _ Case Study _ AWS.txt
file_id: doc296


In [46]:
retrieved_docs_text = [retrieved_file]
context = "\nExtracted documents:\n"
context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)])

final_prompt = RAG_PROMPT_TEMPLATE.format(question=user_query, context=context)

print(final_prompt)

<|system|>
Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:

Extracted documents:
Document 0:::
Amazon DynamoDB is a fully managed, serverless, key-value NoSQL database designed to run high-performance applications at any scale. DynamoDB offers built-in security, continuous backups, automated multi-Region replication, in-memory caching, and data import and export tools.  Learn more »
Amazon Cognito provides an identity store that scales to millions of users, supports social and enterprise identity federation, and offers advanced security features to protect your consumers and business.   Learn more »
Français
Español
more Black engineers and 26% more Hispanic or Latino engineers than industry averages


In [48]:
from langchain_core.output_parsers import JsonOutputParser
answer = llm(final_prompt)
print(answer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Lambda is a serverless compute service offered by Amazon Web Services (AWS). With Lambda, you can run code without provisioning or managing servers, making it a flexible and cost-effective option for processing large amounts of data, automating tasks, or responding to events. Here are some ways you can use Lambda:

1. Process data: You can use Lambda to process large volumes of data without having to manage servers. For example, you could use Lambda to extract insights from log files, transform data between formats, or perform batch computations.

2. Automate tasks: Lambda allows you to automate repetitive tasks, such as sending email notifications, generating reports, or triggering workflows. By automating these tasks, you can free up resources and reduce manual errors.

3. Respond to events: Lambda can be triggered by events from various AWS services, such as Amazon S3 object creations, Amazon Kinesis data streams, or Amazon DynamoDB table updates. This enables you to build event-dri